### The PyCUDA SourceModule function
- The SourceModule function from PyCUDA to compile raw inline CUDA C code into usable kernels that we can launch from Python.
- SourceModule actually compiles code into a CUDA module, this is like a Python module or Windows DLL, only it contains a collection of compiled CUDA code.
- Conclusion: Write a kernel in pure CUDA C and launch it to use a specific number of threads on our GPU

In [1]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray
from pycuda.compiler import SourceModule

# Use a CUDA kernel with SourceModule
# Kernel Function: Multiply a vector by a scalar
# 1. Precede with the __global__ keyword to distinguish the function as a kernel to the compiler
# 2. Declare a void function (Get output values by passing a pointer to some empty chunk of memory that we pass in as a parameter)
# 2a. outvec - output scaled vector, which is a floating-point array pointer
# 2b. scalar - represented with a float (not pointer)
# 2c. vec - input vector, another floating-point array pointer
# 3. Singleton input parameters to a kernel function can be passed in directly from the host without using pointers or allocated device memory.
# 3a. Last chapter: Singleton input parameters to a kernel function can be passed in directly from the host without using pointers or allocated device memory.
# 3b. Identification of each individual thread is given by the threadIdx value, which we retrieve as follows: int i = threadIdx.x
# 3c. threadIdx tells each individual thread its identity, to determine an index for what values should be processed on the input and output data arrays. (This can also be used for assigning particular threads different tasks than others with standard C control flow statements such as if or switch.)
# 4. outvec[i] = scalar*vec[i] - perform scalar multiplication in parallel like we did previously
ker = SourceModule("""
__global__ void scalar_multiply_kernel(float *outvec, float scalar, float *vec)
{
     int i = threadIdx.x;
     outvec[i] = scalar*vec[i];
}
""")

# PyCUDA's get_function - pull out a reference to compiled kernel function from the CUDA module that just compiled with SourceModule
scalar_multiply_gpu = ker.get_function("scalar_multiply_kernel")

# Floating-point array of 512 random vals
testvec = np.random.randn(512).astype(np.float32)

# Copy nto an array in the GPU's global memory using the gpuarray.to_gpu function
testvec_gpu = gpuarray.to_gpu(testvec)

# Allocate a chunk of empty memory to the GPU's global memory using the gpuarray.empty_like function
outvec_gpu = gpuarray.empty_like(testvec_gpu)

# Set scalar value as 2 (Scalar is singleton, don't need to copy the val to GPU, have to typecast properly)
# Set the num of threads to 512 with the block & grid parameters
scalar_multiply_gpu( outvec_gpu, np.float32(2), testvec_gpu, block=(512,1,1), grid=(1,1,1))

# Check whether the output matches with the expected output by using the get function of the gpuarray object
# COmpare via NumPy's allclose
print("Does our kernel work correctly? : {}".format(np.allclose(outvec_gpu.get() , 2*testvec) ))


C:\Users\foonwong\AppData\Local\Temp\ipykernel_16092\4201704412.py:19: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  ker = SourceModule("""


Does our kernel work correctly? : True
